In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
link = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"

In [60]:
df1 = pd.read_csv(link)

In [61]:
df = df1.copy()

In [62]:
df.shape

(1462, 9)

In [63]:
df.head(3)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1


In [64]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [65]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [66]:
df[categorical_columns].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [67]:
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [68]:
df[categorical_columns].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [69]:
df.annual_income = df.annual_income.fillna(0.0)

In [70]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [71]:
list(df.columns)

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score',
 'converted']

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [73]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [74]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [75]:
numerical_columns = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [76]:
df_train[numerical_columns]

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,2,95543.0,3,0.78
1,1,54924.0,6,0.39
2,2,77352.0,2,0.22
3,2,34600.0,2,0.31
4,0,43615.0,2,0.01
...,...,...,...,...
871,5,67314.0,2,0.87
872,6,63996.0,4,0.92
873,1,73702.0,2,0.55
874,1,93341.0,4,0.99


In [77]:
from sklearn.metrics import roc_auc_score

for val in numerical_columns:
    print(val)
    score = roc_auc_score(y_train, df_train[val])
    print('AUC', round(score,3))
    print()

number_of_courses_viewed
AUC 0.764

annual_income
AUC 0.552

interaction_count
AUC 0.738

lead_score
AUC 0.614



In [78]:
numerical_columns, categorical_columns

(['number_of_courses_viewed',
  'annual_income',
  'interaction_count',
  'lead_score'],
 ['lead_source', 'industry', 'employment_status', 'location'])

#### Sol 1: The numerical variable  with has the highest AUC is number_of_courses_viewed with AUC 0.764 .

In [98]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [101]:
val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]

In [102]:
val_auc = roc_auc_score(y_val, y_pred)
round(val_auc,3)

0.817

#### Sol 2: The AUC of this model on the validation dataset is 0.817 and the closest to this is 0.72 <<<



In [ ]:
adfs